01-initial_data_clean.ipynb

first run download_convert.sh to download sql file and convert to csv
- Expand predicates with OR operations into individual predicates
- Convert cuis that are entrez ids into cuis
- Change neg props to the same prop with a negative flag
- Make a separate nodes table

In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
import itertools
import numpy as np

In [2]:
DATA  = 'data/'
SEMMEDDB_PREDICATION_CSV = DATA+"semmedVER42_2020_R_PREDICATION.csv"
EDGES1_CSV = DATA+"edges1.csv"
EDGES2_CSV = DATA+"edges2.csv"
EDGES3_CSV = DATA+"edges3.csv"
NODES1_CSV = DATA+"nodes1.csv"

In [3]:
# read in csv, group rows together with the same subj -> pred -> obj and keep count of number of pmids
# done in chunks
cols = ['SUBJECT_CUI','PREDICATE','OBJECT_CUI','PMID']
gb_cols = ['SUBJECT_CUI','PREDICATE','OBJECT_CUI']
sem_df = pd.DataFrame(columns=cols)
df_iter = pd.read_csv(SEMMEDDB_PREDICATION_CSV, dtype=str, usecols=cols, chunksize=10000000)
for chunk in tqdm(df_iter, total=10):
    chunk.PMID = chunk.PMID.astype("str")
    c = chunk.groupby(gb_cols).PMID.agg(";".join).reset_index()
    sem_df = sem_df.append(c)

In [4]:
# group everything together again
print(len(sem_df))
sem_df = sem_df.groupby(gb_cols).PMID.agg(";".join).reset_index()
print(len(sem_df))

35850848
20925800


In [5]:
print(sem_df.PMID.str.count(";").value_counts()[:5])

0    14039141
1     2904485
2     1168860
3      636552
4      396795
Name: PMID, dtype: int64


In [6]:
sem_df.head()

,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID
0,1,ADMINISTERED_TO,C0007634,24096582
1,1,AFFECTS,C0005935,29798367
2,1,AFFECTS,C0020291,6298464
3,1,AFFECTS,C0028754,19789049
4,1,AFFECTS,C0036421,31505074;31505074


In [7]:
sem_df.to_csv(EDGES1_CSV)

#### in semmedDB some subjects and objects of extracted statements contained the pipe character | as an indicator of multiple concepts in the sentence.

In [8]:
# separate out lines with pipes from the rest
multi_start = sem_df['SUBJECT_CUI'].str.contains('|', regex=False)
multi_end = sem_df['OBJECT_CUI'].str.contains('|', regex=False)
pipe_lines = sem_df[multi_start | multi_end].copy()
sem_df = sem_df[~multi_start & ~multi_end]
print('Rows with multiple subjects or objects {:,}'.format(len(pipe_lines)))
print('Rows with only 1 subject AND only 1 object {:,}'.format(len(sem_df)))

Rows with multiple subjects or objects 2,817,756
Rows with only 1 subject AND only 1 object 18,108,044


In [9]:
pipe_lines.SUBJECT_CUI = pipe_lines.SUBJECT_CUI.str.split('|')
pipe_lines.OBJECT_CUI = pipe_lines.OBJECT_CUI.str.split('|')
pipe_lines.head()

,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID
26,[1],COEXISTS_WITH,"[C0003241, 4099]",24639825;24639825;24639825
39,[1],INHIBITS,"[C0034802, 1956]",22865653
42,[1],INTERACTS_WITH,"[5295, 5296, 8874, 9139, 23368, 54776]",20502503
45,[1],INTERACTS_WITH,"[C0812252, 1398]",20502503
47,[1],INTERACTS_WITH,"[C1335280, 5781]",20502503


In [10]:
# do the combinations
lines = []
for row in tqdm(pipe_lines.itertuples(), total=len(pipe_lines)):
    #print(row)
    a = [row.SUBJECT_CUI, row.OBJECT_CUI]
    c = list(itertools.product(*a))
    lines.extend([{'SUBJECT_CUI':x[0], 'PREDICATE':row.PREDICATE, 'OBJECT_CUI':x[1], 'PMID': row.PMID} for x in c])
expanded_df = pd.DataFrame(lines)

In [11]:
print(len(expanded_df))
expanded_df.head()

6859161


,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID
0,1,COEXISTS_WITH,C0003241,24639825;24639825;24639825
1,1,COEXISTS_WITH,4099,24639825;24639825;24639825
2,1,INHIBITS,C0034802,22865653
3,1,INHIBITS,1956,22865653
4,1,INTERACTS_WITH,5295,20502503


In [12]:
# append the expanded rows to the original df
print(len(sem_df))
sem_df = sem_df.append(expanded_df, sort=True)
print(len(sem_df))

18108044
24967205


In [13]:
# Group rows together again and aggregate pmids
sem_df = sem_df.groupby(gb_cols).PMID.agg(";".join).reset_index()
print(len(sem_df))

23971617


In [16]:
# Not sure why but some SUBJECT_CUI's are empty at this point
sem_df.head()
sem_df.dropna(subset=['SUBJECT_CUI'], inplace=True)
sem_df.head()

In [19]:
# Cache a copy of the current dataframe
sem_df.to_csv(EDGES2_CSV)

In [3]:
sem_df = pd.read_csv(EDGES2_CSV, index_col=0)

/Users/richard/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/richard/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
# change the neg to the same prop without neg and add a neg flag field
idx = sem_df["PREDICATE"].str.startswith("NEG_")
sem_df['NEG'] = False
sem_df.loc[idx, 'NEG'] = True
sem_df.loc[idx, 'PREDICATE'] = sem_df[idx].PREDICATE.str.replace("NEG_", "")
sem_df[sem_df.NEG].head()

In [ ]:
# Cache updated edges with neg flag field
sem_df.to_csv(EDGES3_CSV)

### There are many CUIs that are not actually CUIs. Mike has determined they are (usually/always?) Entrez IDs

In [ ]:
sem_df = pd.read_csv(EDGES3_CSV, index_col=0)

In [49]:
# are going to get them out of this MRSAT file (download from UMLS Metathesaurus)
# I did: cat MRSAT.RRF.a* | gzip > MRSAT.RRF.gz
names = list("abcdefghijklmn")
iter_csv = pd.read_csv("MRSAT.RRF.gz", delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_entrez = dict()
for chunk in tqdm_notebook(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "ENTREZGENE_ID"]
    d = dict(zip(chunk.a, chunk.k))
    umls_entrez.update(d)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
entrez_umls = {v:k for k,v in umls_entrez.items()}
entrez_umls['9987']

'C1415639'

In [51]:
sem_df.SUBJECT_CUI = sem_df.SUBJECT_CUI.map(lambda x:entrez_umls[x] if x in entrez_umls else x)
sem_df.OBJECT_CUI = sem_df.OBJECT_CUI.map(lambda x:entrez_umls[x] if x in entrez_umls else x)

In [ ]:
noncdf = sem_df[~sem_df.SUBJECT_CUI.str.startswith("C")]
print(len(noncdf))

In [ ]:
# dump everything that doesn't starts with a "C"
print(len(sem_df))
sem_df = sem_df[sem_df.SUBJECT_CUI.str.startswith("C")]
sem_df = sem_df[sem_df.OBJECT_CUI.str.startswith("C")]
print(len(sem_df))

In [ ]:
# sem_df.to_csv(EDGES2_CSV)
# sem_df = pd.read_csv(EDGES2_CSV, index_col=0)

In [52]:
### Make a nodes table
conso = pd.read_csv("MRCONSO_ENG.RRF.gz", delimiter="|", index_col=None, names = list("abcdefghijklmnopqrs"))
conso = conso[(conso['c'] == "P") & (conso['e'] == "PF")]
conso.head()

21694


In [53]:
# dump everything that doesn't starts with a "C"
print(len(sem_df))
sem_df = sem_df[sem_df.SUBJECT_CUI.str.startswith("C")]
sem_df = sem_df[sem_df.OBJECT_CUI.str.startswith("C")]
print(len(sem_df))

22322647
22280924


In [3]:
# sem_df.to_csv(EDGES2_CSV)
# sem_df = pd.read_csv(EDGES2_CSV, index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
### Make a nodes table
conso = pd.read_csv("MRCONSO_ENG.RRF.gz", delimiter="|", index_col=None, names = list("abcdefghijklmnopqrs"))
conso = conso[(conso['c'] == "P") & (conso['e'] == "PF")]
conso.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0,NaN
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN
32,C0000052,ENG,P,L0000052,PF,S0007584,N,A0016535,NaN,M0023173,D015061,MSH,MH,D015061,"1,4-alpha-Glucan Branching Enzyme",0,N,256.0,NaN
33,C0000052,ENG,P,L0000052,PF,S0007584,N,A18020417,NaN,N0000168345,NaN,NDFRT,PT,N0000168345,"1,4-alpha-Glucan Branching Enzyme",0,N,256.0,NaN


In [7]:
node_label = dict(zip(conso.a, conso.o))
print(len(node_label))

3654581


In [8]:
nodes = set(sem_df.SUBJECT_CUI) | set(sem_df.OBJECT_CUI)
print(len(nodes))

271725


In [9]:
nodes = pd.DataFrame({"ID":x, "LABEL": node_label.get(x)} for x in nodes)
nodes = nodes.dropna()
print(len(nodes))
nodes.head()

259227


,ID,LABEL
0,C0061133,gastrin releasing peptide (14-27)
1,C1523610,"regulation of tube length, open tracheal system"
2,C0312636,Antibody to hepatitis E virus
3,C1532578,mL/cm H2O
4,C0539817,cytochrome p30


In [10]:
nodes.to_csv(NODES1_CSV)